# Images cleanup utility
This notebook fixes data consistency issues where image file extensions on disk do not match the extensions specified in their source urls (e.g., a .webp file saved with a .jpg extension).

### Dependencies and namespaces
Imports ef core sqlite and loads local project dlls for data access and entity models.

In [ ]:
#r "nuget: Microsoft.EntityFrameworkCore.Sqlite"
#r "..\Japlayer.Data\bin\Debug\net8.0\Japlayer.Data.dll"
#r "..\Japlayer.WinUI\bin\Debug\net8.0-windows10.0.19041.0\Japlayer.dll"

using Japlayer.Data.Context;
using Japlayer.Data.Entities;
using Japlayer.Data.Services;
using Japlayer.Models;
using Microsoft.EntityFrameworkCore;
using System.IO;
using System.Text.Json;

### Configuration and database initialization
Loads application settings from json to locate the sqlite database and initialize the databasecontext.

In [ ]:
var settingsFilePath = @"..\japlayer.settings.json";
string jsonContent = File.ReadAllText(settingsFilePath);
var options = new JsonSerializerOptions { PropertyNameCaseInsensitive = true };
AppSettings settings = JsonSerializer.Deserialize<AppSettings>(jsonContent, options);

var optionsBuilder = new DbContextOptionsBuilder<DatabaseContext>();
optionsBuilder.UseSqlite($"Data Source={settings.SqliteDatabasePath}");

var context = new DatabaseContext(optionsBuilder.Options);
Console.WriteLine($"✓ Initialized context for: {settings.SqliteDatabasePath}");

### Identify inconsistent extensions
Filters records where the url extension and filepath extension do not match.

In [ ]:
var candidates = await context.MediaImages
    .Where(m => m.Url.Contains(".") && m.Filepath.Contains("."))
    .ToListAsync();

var mismatched = candidates
    .Where(m => Path.GetExtension(m.Url).ToLower() != 
                Path.GetExtension(m.Filepath).ToLower())
    .ToList();

mismatched

### Integrity validation
Defines a hashing function to verify if two files are bit-for-bit identical before performing disk operations.

In [ ]:
using System.Security.Cryptography;

public bool AreFilesIdentical(string path1, string path2)
{
    if (!File.Exists(path1) || !File.Exists(path2)) return false;
    if (new FileInfo(path1).Length != new FileInfo(path2).Length) return false;

    using var sha256 = SHA256.Create();
    using var stream1 = File.OpenRead(path1);
    using var stream2 = File.OpenRead(path2);

    byte[] hash1 = sha256.ComputeHash(stream1);
    byte[] hash2 = sha256.ComputeHash(stream2);

    return StructuralComparisons.StructuralEqualityComparer.Equals(hash1, hash2);
}

### Database update and file migration
Runs a transaction to update metadata foreign keys, replace the mediaimage records, and move or delete the physical files on disk.

In [ ]:
using (var transaction = await context.Database.BeginTransactionAsync())
{
    try
    {
        foreach (var img in mismatched)
        {
            context.ChangeTracker.Clear();

            string oldRelativePath = img.Filepath;
            string newRelativePath = Path.ChangeExtension(oldRelativePath, ".webp");
            
            string absoluteOldPath = Path.Combine(settings.ImagePath, oldRelativePath);
            string absoluteNewPath = Path.Combine(settings.ImagePath, newRelativePath);

            var relatedMetadata = await context.MediaMetadata
                .Where(m => m.Cover == oldRelativePath || m.Thumbnail == oldRelativePath)
                .ToListAsync();

            foreach (var meta in relatedMetadata)
            {
                if (meta.Cover == oldRelativePath) meta.Cover = newRelativePath;
                if (meta.Thumbnail == oldRelativePath) meta.Thumbnail = newRelativePath;
            }

            context.MediaImages.Add(new MediaImage { MediaId = img.MediaId, Url = img.Url, Filepath = newRelativePath });
            context.MediaImages.Remove(new MediaImage { Filepath = oldRelativePath });
            
            await context.SaveChangesAsync();

            if (File.Exists(absoluteNewPath))
            {
                if (File.Exists(absoluteOldPath)) File.Delete(absoluteOldPath);
            }
            else if (File.Exists(absoluteOldPath))
            {
                File.Move(absoluteOldPath, absoluteNewPath);
            }
        }
        await transaction.CommitAsync();
    }
    catch (Exception)
    {
        await transaction.RollbackAsync();
        throw;
    }
}